In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

dataframe1_path = '1_merged.csv'
dataframe1 = pd.read_csv(dataframe1_path)

dataframe2_path = '2_merged.csv'
dataframe2 = pd.read_csv(dataframe2_path)

dataframe3_path = '3_merged.csv'
dataframe3 = pd.read_csv(dataframe3_path)



In [ ]:
from sklearn.neighbors import LocalOutlierFactor


combined_data1 = dataframe1[['Timestamp', 'SBT Z-axis', 'Magnetic Z-axis']]
combined_data2 = dataframe2[['Timestamp', 'SBT Z-axis', 'Magnetic Z-axis']]
combined_data3 = dataframe3[['Timestamp', 'SBT Z-axis', 'Magnetic Z-axis']]


combined_data = pd.concat([combined_data1, combined_data2,combined_data3])

data = combined_data[['SBT Z-axis', 'Magnetic Z-axis']]


lof = LocalOutlierFactor(n_neighbors=50, contamination=0.35)


combined_data['anomaly'] = lof.fit_predict(data)

cleaned_data = combined_data[combined_data['anomaly'] == 1]

In [ ]:
from sklearn.model_selection import train_test_split


X = cleaned_data[['Magnetic Z-axis']].values 
y = cleaned_data['SBT Z-axis'].values  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense


def build_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=input_dim))
    for _ in range(4): 
        model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  
    return model

def build_linear_model():
    return LinearRegression()


In [ ]:


from sklearn.model_selection import KFold, cross_val_score
def evaluate_model(X, y, build_model_func, model_type='mlp', n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=77)
    mae_scores = []
    mse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        if model_type == 'mlp':
            model = build_model_func(X_train.shape[1])
            optimizer = Adam(learning_rate=1e-3)
            model.compile(optimizer=optimizer, loss='mean_absolute_error')
            reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=1)
            checkpoint = ModelCheckpoint('Z_best_model.h5', monitor='loss', save_best_only=True, verbose=1)
            early_stopping = EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=1, restore_best_weights=True)
            model.fit(X_train, y_train, epochs=500, callbacks=[reduce_lr, checkpoint, early_stopping], verbose=0)
            predictions = model.predict(X_test).flatten()
        else:
            model = build_model_func()
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        mae = mean_absolute_error(y_test, predictions)
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)

        mae_scores.append(mae)
        mse_scores.append(mse)
        r2_scores.append(r2)

    print(f'Mean Absolute Error: {np.mean(mae_scores)}')
    print(f'Mean Squared Error: {np.mean(mse_scores)}')
    print(f'R^2 Score: {np.mean(r2_scores)}')

In [ ]:
# Evaluate MLP Model
evaluate_model(X, y, build_mlp_model, model_type='mlp')


In [ ]:
# Evaluate Linear model
evaluate_model(X, y, build_linear_model, model_type='linear')